In [2]:
!pip install peft
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import get_peft_model
from peft import LoraConfig, TaskType
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [3]:
competitionData = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')
externalData = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_01.csv')
competitionData = competitionData.drop(['id', 'prompt_id'], axis=1)
externalData = externalData.drop(['source', 'fold'], axis=1)
competitionData = competitionData.rename(columns={'generated': 'label'})
finalData = pd.concat([competitionData, externalData])

# Load the pre-trained BERT model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased')
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Split the data into training and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    finalData['text'].tolist(),
    finalData['label'].tolist(),
    test_size=0.2,
    random_state=42
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Tokenize the training and test sets
tokenized_train = tokenizer(
    train_texts,
    padding=True,
    truncation=True,
    return_tensors='pt'
)

tokenized_test = tokenizer(
    test_texts,
    padding=True,
    truncation=True,
    return_tensors='pt'
)

In [5]:
# Create PyTorch datasets for training and test sets
class CustomDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

In [6]:
# Training dataset
train_dataset = CustomDataset(
    input_ids=tokenized_train['input_ids'],
    attention_mask=tokenized_train['attention_mask'],
    labels=train_labels
)

# Test dataset
test_dataset = CustomDataset(
    input_ids=tokenized_test['input_ids'],
    attention_mask=tokenized_test['attention_mask'],
    labels=test_labels
)

In [7]:
# Configure PEFT model
peft_config = LoraConfig(task_type=TaskType.SEQ_CLS, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)
model = get_peft_model(model, peft_config)

In [12]:
# Training arguments
training_args = TrainingArguments(
    output_dir="/kaggle/working/",
    learning_rate=1e-3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [13]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Move model to GPU for training
trainer.model.to('cuda')

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.034700,0.053801
2,0.004800,0.016471


TrainOutput(global_step=6928, training_loss=0.028100146646944775, metrics={'train_runtime': 2567.2739, 'train_samples_per_second': 21.586, 'train_steps_per_second': 2.699, 'total_flos': 1.463155732819968e+16, 'train_loss': 0.028100146646944775, 'epoch': 2.0})

In [16]:
model.save_pretrained("/kaggle/working/")

In [18]:
from huggingface_hub import notebook_login

notebook_login()
model.push_to_hub("aiVshuman_bert_2epochs")

adapter_model.safetensors:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/MonkeyDdonut/aiVshuman_bert_2epochs/commit/4fd51207a9b760b92096f67b89583dcae78aea22', commit_message='Upload model', commit_description='', oid='4fd51207a9b760b92096f67b89583dcae78aea22', pr_url=None, pr_revision=None, pr_num=None)